In [115]:
import torch
from kan import *
from my_functions import calculate_signature, calculate_signature_over_all_surface, format_signature
import plotly.graph_objects as go


In [116]:
mesh_path = "data/bunny_curvs.ply"
sampled_index = 30000
dataset = create_dataset_from_mesh(mesh_path=mesh_path,sampled_index=sampled_index, show_sample=True, ball_radius=0.06, down_sampling_ratio=0.8)
x = torch.tensor([[0.0], [0.0]], requires_grad=True).T
dataset['train_input'].shape, dataset['train_label'].shape

(torch.Size([149, 2]), torch.Size([149, 1]))

In [117]:
experiment_num = 10
signatures = torch.zeros(experiment_num, 6)
signatures2 = torch.zeros(experiment_num, 6)
dataset_sampled_points = []
grid_range = 1.0
ball_radius = 0.07
lr = 0.1
for i in range(experiment_num):
    dataset = create_dataset_from_mesh(mesh_path=mesh_path,sampled_index=sampled_index, show_sample=False, ball_radius=ball_radius, seed=i, down_sampling_ratio=0.8)

    model = KAN(width=[2,5,1], grid=3, k=5, grid_eps=1.0, seed=0, grid_range=[-grid_range,grid_range], noise_scale=0.1, noise_scale_base=1.0, base_fun=torch.nn.SiLU(), learn_rotation_mat=False)
    model.train(dataset, opt="LBFGS", steps=20,lr=lr, lamb=100.0, lamb_l1=1.0, lamb_entropy=1.0, lamb_coef=0.0, lamb_coefdiff=0.0, sglr_avoid=True)
    model = model.prune(threshold=3e-3, mode='manual', active_neurons_id=[[0,1],[0]])
    model.train(dataset, opt="LBFGS", steps=50,lr=lr, sglr_avoid=True)
     # manual mode
    # fix all
    model.fix_symbolic(0,0,0,'x^4')
    model.fix_symbolic(0,1,0,'x^3')
    model.fix_symbolic(1,0,0,'x^2')

    # model.fix_symbolic(1,1,0, 'x')
    # model.fix_symbolic(0,0,1,'x^5')
    # model.fix_symbolic(0,1,1,'x')
    # model.fix_symbolic(1,1,0,'x^5')
    # model.fix_symbolic(1,1,0,'1')
    # model.fix_symbolic(1,1,0,'x')
    # model.fix_symbolic(2,0,0,'1')

    # lib = ['x','x^2', 'x^3', 'x^4', 'sqrt']
    # model.auto_symbolic(lib=lib)
    model.learn_rotation_mat = False
    model.train(dataset, opt="LBFGS", steps=50, lr=lr,sglr_avoid=True, lamb_coef=1.0, lamb_coefdiff=1.0, loss_fn=gaussian_weighted_mse)

    print(model.symbolic_formula()[0][0])
    signature = calculate_signature(model, x) # only calc sig in the origin
    # signature2 = calculate_signature_over_all_surface(model, torch.tensor(dataset['train_input'], requires_grad=True)) # calc sig over all surface and take the mean
    print("exp ", i, "signature ", signature)
    signatures[i] = signature
    # signatures2[i] = signature2
    curr_sampled_points = torch.concatenate((dataset['test_input'], model(dataset['test_input'])), dim=1)
    dataset_sampled_points.append(curr_sampled_points)


train loss: 3.47e-02 | test loss: 3.45e-02 | reg: 2.64e+00 : 100%|██| 20/20 [00:02<00:00,  8.60it/s]
train loss: 9.73e-04 | test loss: 4.74e-03 | reg: 1.73e+00 : 100%|██| 50/50 [00:02<00:00, 17.28it/s]


Best value at boundary.
r2 is 0.5996698141098022
r2 is not very high, please double check if you are choosing the correct symbolic function.
r2 is 0.753095805644989
r2 is not very high, please double check if you are choosing the correct symbolic function.
Best value at boundary.
r2 is 0.9317346215248108


train loss: 2.62e-03 | test loss: 3.91e-03 | reg: 3.16e+00 : 100%|██| 50/50 [00:00<00:00, 52.18it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



40689634.4*(-x_1 - 0.01)**8 - 0.03
exp  0 signature  tensor([[  -2.5020,    2.1661,  353.8773,    4.7416, -339.1986,  -41.3147]],
       grad_fn=<StackBackward0>)


train loss: 3.35e-02 | test loss: 3.56e-02 | reg: 2.50e+00 : 100%|██| 20/20 [00:02<00:00,  8.87it/s]
train loss: 1.06e-03 | test loss: 5.69e-03 | reg: 1.69e+00 : 100%|██| 50/50 [00:02<00:00, 19.57it/s]


...

30928419.76*(-x_1 - 0.01)**8 - 0.04
exp  8 signature  tensor([[  -2.5815,    1.8937,  369.4449,    3.8281, -293.8260,  -35.1224]],
       grad_fn=<StackBackward0>)


train loss: 3.53e-02 | test loss: 3.73e-02 | reg: 2.67e+00 : 100%|██| 20/20 [00:02<00:00,  9.50it/s]
train loss: 9.24e-04 | test loss: 5.87e-03 | reg: 1.89e+00 : 100%|██| 50/50 [00:02<00:00, 18.79it/s]


Best value at boundary.
r2 is 0.6057252883911133
r2 is not very high, please double check if you are choosing the correct symbolic function.
r2 is 0.6532996296882629
r2 is not very high, please double check if you are choosing the correct symbolic function.
Best value at boundary.
r2 is 0.9439144730567932


train loss: 2.52e-03 | test loss: 4.73e-03 | reg: 3.52e+00 : 100%|██| 50/50 [00:00<00:00, 53.78it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



41070177.8*(-x_1 - 0.01)**8 - 0.03
exp  9 signature  tensor([[  -2.2877,    1.1071,  342.4861,    2.1209, -175.8061,  -17.2139]],
       grad_fn=<StackBackward0>)


In [118]:
# plot the sampled points in the same plot but different colors
fig = go.Figure()
for i in range(len(dataset_sampled_points)):
    curr_sampled_points = dataset_sampled_points[i].detach()
    fig.add_trace(go.Scatter3d(x=curr_sampled_points[:,0], y=curr_sampled_points[:,1], z=curr_sampled_points[:,2], mode='markers', marker=dict(size=3), name="exp "+str(i)))
fig.show()

In [119]:
signatures

tensor([[  -2.5020,    2.1661,  353.8773,    4.7416, -339.1986,  -41.3147],
        [  -2.6250,    2.2795,  364.3163,    4.3331, -348.4916,  -39.3345],
        [  -2.4983,    1.9826,  354.9867,    3.2650, -308.9487,  -27.6269],
        [  -2.3159,    1.4729,  344.1118,    2.8172, -236.7661,  -22.5800],
        [  -2.2026,    2.0729,  351.8073,    5.3110, -377.2722,  -39.9709],
        [  -2.6199,    2.4757,  371.6575,    5.4157, -390.6307,  -49.5021],
        [  -2.5067,    1.8337,  360.1658,    3.8700, -286.4154,  -34.4064],
        [  -2.3596,    2.4873,  381.3403,    5.8843, -461.1435,  -47.4398],
        [  -2.5815,    1.8937,  369.4449,    3.8281, -293.8260,  -35.1224],
        [  -2.2877,    1.1071,  342.4861,    2.1209, -175.8061,  -17.2139]],
       grad_fn=<CopySlices>)

In [120]:
 #discard all nan and 0
signatures = signatures[~torch.isnan(signatures).any(dim=1)]
signatures = signatures[~(signatures == 0).all(dim=1)]

In [121]:
mean_signature = torch.mean(signatures, dim=0)
std_signature = torch.std(signatures, dim=0)
print("mean signature ", mean_signature)
print("std signature ", std_signature)



mean signature  tensor([  -2.4499,    1.9772,  359.4194,    4.1587, -321.8499,  -35.4512],
       grad_fn=<MeanBackward1>)
std signature  tensor([ 0.1486,  0.4325, 12.4039,  1.2110, 80.9604, 10.3909],
       grad_fn=<StdBackward0>)


In [122]:
#discard all nan and 0
# signatures2 = signatures2[~torch.isnan(signatures2).any(dim=1)]
# signatures2 = signatures2[~(signatures == 0).all(dim=1)]
# mean_signature2 = torch.mean(signatures2, dim=0)
# std_signature2 = torch.std(signatures2, dim=0)
# print("mean signature ", mean_signature2)
# print("std signature ", std_signature2)


In [123]:
# remove outliers that are 2 stds away from the mean
outliers = torch.abs(signatures - mean_signature) > 2*std_signature
signatures_without_outliers = signatures[~outliers.any(dim=1)]
mean_signature = torch.mean(signatures_without_outliers, dim=0)
std_signature = torch.std(signatures_without_outliers, dim=0)
print("mean signature ", mean_signature)
print("std signature ", std_signature)
print(signatures_without_outliers.shape)
print(signatures_without_outliers)



mean signature  tensor([  -2.4679,    2.0738,  361.3009,    4.3851, -338.0770,  -37.4775],
       grad_fn=<MeanBackward1>)
std signature  tensor([ 0.1456,  0.3246, 11.5440,  1.0359, 66.4198,  8.6760],
       grad_fn=<StdBackward0>)
torch.Size([9, 6])
tensor([[  -2.5020,    2.1661,  353.8773,    4.7416, -339.1986,  -41.3147],
        [  -2.6250,    2.2795,  364.3163,    4.3331, -348.4916,  -39.3345],
        [  -2.4983,    1.9826,  354.9867,    3.2650, -308.9487,  -27.6269],
        [  -2.3159,    1.4729,  344.1118,    2.8172, -236.7661,  -22.5800],
        [  -2.2026,    2.0729,  351.8073,    5.3110, -377.2722,  -39.9709],
        [  -2.6199,    2.4757,  371.6575,    5.4157, -390.6307,  -49.5021],
        [  -2.5067,    1.8337,  360.1658,    3.8700, -286.4154,  -34.4064],
        [  -2.3596,    2.4873,  381.3403,    5.8843, -461.1435,  -47.4398],
        [  -2.5815,    1.8937,  369.4449,    3.8281, -293.8260,  -35.1224]],
       grad_fn=<IndexBackward0>)


In [124]:
# calculate std/mean to see the variation of the signature
std_over_mean = abs(std_signature/mean_signature)
print("std/mean ", std_over_mean)

std/mean  tensor([0.0590, 0.1565, 0.0320, 0.2362, 0.1965, 0.2315],
       grad_fn=<AbsBackward0>)
